In [ ]:
# Task B
!pip install transformers torch accelerate
!huggingface-cli login
!huggingface-cli whoami
from transformers import AutoTokenizer
import transformers
import torch
model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

from transformers import pipeline
# initialization of pipeline
llama_pipeline = pipeline(
    "text-generation",
    model = model,
    torch_dtype=torch.float16,
    device_map="auto",
)
# make sure the end of response is a full sentence
def complete_sentence(response:str)-> str:
  for i in ['.','?','!']:
    last = response.rfind(i) # find last position of punctuation
    if last != -1:
      return response[:last+1] # return response end with full sentence
  return response

# generate response
def get_llama_response1(prompt: str, old_conversation: str)-> str:
  full_prompt = old_conversation + "\n only anwser this question:" + prompt
  sequences = llama_pipeline(
      full_prompt,
      do_sample=True,
      top_k=12,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_new_tokens=256,
      temperature=1.1
  )
  generated_content = sequences[0]['generated_text']
  response = generated_content[len(full_prompt):].strip() # cut off unnecessary text
  return response

old_conversation = ""

# conversation loop
while True:
  user_input = input("You: ")
  if user_input.lower() in ["bye", "quit", "exit"]:
    print("Chatbot: Goodbye!")
    break
  response = get_llama_response1(user_input, old_conversation)
  response = complete_sentence(response)
  old_conversation += f"user:\n{user_input} chatbot:\n{response}" # append conversation history
  print("Chatbot:", response,"\n")